## Metaprogramming

Dynamic attributes and properties

In [ ]:
# osconfeed.py
from urllib.request import urlopen
import warnings
import os
import json
import ssl

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'
gcontext = ssl.SSLContext(ssl.PROTOCOL_TLSv1)

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL, context=gcontext) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())
            
        with open(JSON) as fp:
            return json.load(fp)

feed = load()
sorted(feed['Schedule'].keys())

In [3]:
import json
with open('data/osconfeed.json') as fp:
    feed = json.load(fp)
    
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [10]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [14]:
feed['Schedule']['speakers'][356]['name']

'Carina C. Zona'

In [15]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [16]:
feed['Schedule']['speakers'][356]['serial']

141590

In [17]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [22]:
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

### Exploring JSON-like data with dynamic attributes

In [24]:
import json

def loadf():
    with open('data/osconfeed.json') as fp:
        feed = json.load(fp)
    return feed
    
raw_feed = loadf()

In [27]:
# explore0.py: turn a JSON dataset into a FrozenJSON
from collections import abc

class FrozenJSON:
    """
    A read-only facade for navigating a JSON-like object
    using attribute notation
    """
    
    def __init__(self, mapping):
        self.__data = dict(mapping)
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
        
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

In [29]:
feed = FrozenJSON(raw_feed)
len(feed.Schedule.speakers)

357

In [30]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [31]:
for key, value in sorted(feed.Schedule.items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [32]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [34]:
talk = feed.Schedule.events[40]
talk

In [35]:
type(talk)

__main__.FrozenJSON

In [36]:
talk.name

'There *Will* Be Bugs'

In [37]:
talk.speakers

[3471, 5199]

In [38]:
talk.flavor

KeyError: 'flavor'

In [15]:
# explore1.py
from collections import abc

class FrozenJSON:
    """
    A read-only facade for navigating a JSON-like object
    using attribute notation
    """
    
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
        
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return  cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

In [16]:
# explore2.py: using __new__ instead of build to construct new objects
from collections import abc

class FrozenJSON:
    
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value
            
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])